In [3]:
from arango import ArangoClient
import pandas as pd 

In [10]:
client = ArangoClient(hosts="http://0.0.0.0:8529/")

db = client.db("AeropuertoDB", username="root", password="7ey7sjN4loZXmxYS")

In [342]:
#Importar las bases de datos

aeropuerto_info = pd.read_csv("DataFrames/Info_Basica_Aeropuertos.csv").iloc[:,1:]
arrivos = pd.read_csv("DataFrames/arrivos.csv").iloc[:,1:]
aerolineas = pd.read_csv("DataFrames/Aerolineas.csv").iloc[:,1:]
aerolineas.fillna("NO",inplace=True)
aerolineas.drop_duplicates(subset=['IATA'],inplace=True)
aerolineas = aerolineas[aerolineas['IATA'].str.len() <= 2]
arrivos = arrivos.applymap(str)
arrivos.drop_duplicates(inplace=True,subset=['Numero de Vuelo'])
arrivos["cod origen"] = arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)
arrivos['cod destino'] = arrivos.apply(lambda x: x['Destino'].split()[-1].replace('(','').replace(')',''), axis=1)
aeropuerto_info['cod aero'] = aeropuerto_info.apply(lambda x: x['Nombre'].split()[-1].replace('(','').replace(')',''), axis=1)
arrivos['cod vuelo'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2]+a[3:])
arrivos['IATA'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2])

In [311]:
aeropuerto_info.head()

,Nombre,Ubicación,Timezone,Teléfono,cod aero
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD
1,San Salvador Cuscatlan Airport (SAL),"RN-5, San Salvador, El Salvador",(UTC/GMT -6),+503 2349 9455,SAL
2,Panama City Albrook Marcos A. Gelabert Airport...,"Avenida Canfield, Panamá, Panama",(UTC/GMT -5),+507 501-9272,PAC
3,Guatemala City La Aurora Airport (GUA),"Guatemala City, Guatemala",(UTC/GMT -6),+502 2321 5000,GUA
4,Puerto Rico Airport (PUR),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,PUR


In [316]:
arrivos.head()

,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo
0,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,SDQ,5X402
1,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,SDQ,AV208
2,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,SDQ,CM107
3,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,SDQ,TK9551
4,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,SDQ,KL3056


In [218]:
aeropuerto_info_db = db.create_collection("aeropuerto")
arrivos_db = db.create_collection("arrivos")
aerolineas_db = db.create_collection("aerolineas")

In [123]:
aeropuerto_info_db.add_hash_index(fields=["Nombre"], unique=True)

{'id': '30177',
 'fields': ['Nombre'],
 'type': 'hash',
 'name': 'idx_1780066938263699456',
 'deduplicate': True,
 'sparse': False,
 'unique': True,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [223]:
for row in range(aeropuerto_info.shape[0]):
        
    aeropuerto_info_db.insert({'_key':aeropuerto_info.iloc[row,0][-4:-1],
                               aeropuerto_info.columns[0]:aeropuerto_info.iloc[row,0],
                               aeropuerto_info.columns[1]:aeropuerto_info.iloc[row,1],
                               aeropuerto_info.columns[2]:aeropuerto_info.iloc[row,2],
                               aeropuerto_info.columns[3]:aeropuerto_info.iloc[row,3]})

In [122]:
arrivos_db.add_hash_index(fields=["Numero de Vuelo"], unique=False)

{'id': '30173',
 'fields': ['Numero de Vuelo'],
 'type': 'hash',
 'name': 'idx_1780066933900574720',
 'deduplicate': True,
 'sparse': False,
 'unique': False,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [241]:
for row in range(arrivos.shape[0]):
    
    arrivos_db.insert({ '_key': arrivos.iloc[row,2][0:2]+arrivos.iloc[row,2][3:],
                       arrivos.columns[0]:arrivos.iloc[row,0],
                       arrivos.columns[1]:arrivos.iloc[row,1],
                       arrivos.columns[2]:arrivos.iloc[row,2],
                       arrivos.columns[3]:arrivos.iloc[row,3],
                       arrivos.columns[4]:arrivos.iloc[row,4]
                      })

In [202]:
aerolineas_db.add_hash_index(fields=["IATA"], unique=True)

{'id': '60574',
 'fields': ['IATA'],
 'type': 'hash',
 'name': 'idx_1780068468941389824',
 'deduplicate': True,
 'sparse': False,
 'unique': True,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [242]:
for row in range(aerolineas.shape[0]):
    
    aerolineas_db.insert({'_key':aerolineas.iloc[row,0].lower(),
                          aerolineas.columns[0]:aerolineas.iloc[row,0],
                          aerolineas.columns[1]:aerolineas.iloc[row,1],
                          aerolineas.columns[2]:aerolineas.iloc[row,2],
                          aerolineas.columns[3]:aerolineas.iloc[row,3],
                          aerolineas.columns[4]:aerolineas.iloc[row,4]
                      })

In [329]:
# Get the API wrapper for graph "school".
ae = db.create_graph('Aeropuerto')

In [330]:
aero_viajes = ae.create_edge_definition(
    edge_collection='vuelo',
    from_vertex_collections=['aeropuerto'],
    to_vertex_collections=['arrivos'])

# Edge collections have a similar interface as standard collections.

for i in range(len(s1)):
    aero_viajes.insert({
        '_key': f'{s1["cod aero"][i]}-{s1["cod vuelo"][i]}',
        '_from': f'aeropuerto/{s1["cod aero"][i]}',
        '_to': f'arrivos/{s1["cod vuelo"][i]}'
    })
    


In [314]:
arrivos_dest = arrivos.rename(columns={"cod origen":'cod aero'})
s1 = pd.merge(aeropuerto_info, arrivos_dest, how='inner', on=['cod aero'])

In [306]:
arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)

0        MIA
1        BOG
2        PTY
3        PTY
4        PTY
        ... 
15922    MCO
15923    SJO
15924    SJO
15925    DTW
15926    DTW
Length: 10082, dtype: object

In [315]:
s1

,Nombre,Ubicación,Timezone,Teléfono,cod aero,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod destino,cod vuelo
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,UA 1285,7:40,10:00,IAH,UA1285
1,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,AV 2312,7:40,10:00,IAH,AV2312
2,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,NZ 6850,7:40,10:00,IAH,NZ6850
3,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,UA 1863,9:19,nan,IAH,UA1863
4,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,NZ 6856,9:19,nan,IAH,NZ6856
...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Toronto Pearson Airport (YYZ),"Silver Dart Drive 6301, L5P 1B2 Mississauga, ...",(UTC/GMT -5),+1 416-247-7678,YYZ,Toronto [YYZ],Phoenix Sky Harbor Airport (PHX),AC 1721,9:15,11:05,PHX,AC1721
2072,Toronto Pearson Airport (YYZ),"Silver Dart Drive 6301, L5P 1B2 Mississauga, ...",(UTC/GMT -5),+1 416-247-7678,YYZ,Toronto [YYZ],Miami Airport (MIA),AC 1200,11:00,13:48,MIA,AC1200
2073,Toronto Pearson Airport (YYZ),"Silver Dart Drive 6301, L5P 1B2 Mississauga, ...",(UTC/GMT -5),+1 416-247-7678,YYZ,Toronto [YYZ],Miami Airport (MIA),AA 2589,6:00,nan,MIA,AA2589
2074,Toronto Pearson Airport (YYZ),"Silver Dart Drive 6301, L5P 1B2 Mississauga, ...",(UTC/GMT -5),+1 416-247-7678,YYZ,Toronto [YYZ],Miami Airport (MIA),BA 5760,6:00,nan,MIA,BA5760


In [327]:
s1["cod aero"][1]

'ORD'

In [331]:
arrivos_dest = arrivos.rename(columns={"cod destino":'cod aero'})
s2 = pd.merge(aeropuerto_info, arrivos_dest, how='inner', on=['cod aero'])

In [339]:
arrivos

,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo,codigo iata
0,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,SDQ,5X402,5X
1,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,SDQ,AV208,AV
2,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,SDQ,CM107,CM
3,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,SDQ,TK9551,TK
4,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,SDQ,KL3056,KL
...,...,...,...,...,...,...,...,...,...
15922,Orlando [MCO],Miami Airport (MIA),IB 4728,9:45,nan,MCO,MIA,IB4728,IB
15923,San José [SJO],Miami Airport (MIA),AA 1204,6:00,10:28,SJO,MIA,AA1204,AA
15924,San José [SJO],Miami Airport (MIA),IB 4551,6:00,10:28,SJO,MIA,IB4551,IB
15925,Detroit [DTW],Miami Airport (MIA),DL 1468,8:00,10:40,DTW,MIA,DL1468,DL


In [334]:
for i in range(len(s2)):
    aero_viajes.insert({
        '_key': f'{s2["cod aero"][i]}--{s2["cod vuelo"][i]}',
        '_from': f'aeropuerto/{s2["cod aero"][i]}',
        '_to': f'arrivos/{s2["cod vuelo"][i]}'
    })
    

In [343]:
s3 = pd.merge(aerolineas, arrivos, how='inner', on=['IATA'])

In [344]:
s3

,IATA,IACO,Aerolinea,Llamada de Identificación,País,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo
0,PR,BOI,2GO,ABAIR,Philippines,Manila [MNL],San Francisco Airport (SFO),PR 104,2:00,nan,MNL,SFO,PR104
1,4Y,OCN,Eurowings Discover,EUROWINGS DISCOVER,Germany,Frankfurt [FRA],Orlando Airport (MCO),4Y 68,14:19,18:09,FRA,MCO,4Y68
2,6E,IGO,IndiGo,IFLY,India,Istanbul [IST],New York John F. Kennedy Airport (JFK),6E 4235,20:32,23:46,IST,JFK,6E4235
3,6E,IGO,IndiGo,IFLY,India,Istanbul [IST],San Francisco Airport (SFO),6E 4272,8:25,12:29,IST,SFO,6E4272
4,GB,ABX,ABX Air,ABEX,United States,Covington [CVG],Houston George Bush Intercontinental Airport (...,GB 2024,5:30,07:13,CVG,IAH,GB2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,WS,WJA,WestJet,WESTJET,Canada,Detroit [DTW],Phoenix Sky Harbor Airport (PHX),WS 7689,16:25,17:17,DTW,PHX,WS7689
9999,WS,WJA,WestJet,WESTJET,Canada,"Calgary, AB [YYC]",Phoenix Sky Harbor Airport (PHX),WS 1536,18:30,nan,YYC,PHX,WS1536
10000,WS,WJA,WestJet,WESTJET,Canada,Seattle [SEA],Phoenix Sky Harbor Airport (PHX),WS 6754,19:50,22:15,SEA,PHX,WS6754
10001,WS,WJA,WestJet,WESTJET,Canada,Atlanta [ATL],Miami Airport (MIA),WS 7777,7:35,nan,ATL,MIA,WS7777


In [346]:
aerolineas_viajes = ae.create_edge_definition(
    edge_collection='aa',
    from_vertex_collections=['arrivos'],
    to_vertex_collections=['aerolineas'])

# Edge collections have a similar interface as standard collections.

for i in range(len(s1)):
    aerolineas_viajes.insert({
        '_key': f'{s3["IATA"][i]}-{s3["cod vuelo"][i]}',
        '_from': f'arrivos/{s3["cod vuelo"][i]}',
        '_to': f'aerolineas/{s3["IATA"][i].lower()}'
    })